## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-18-52-00 +0000,wsj,Trump Says U.S. Will Take Nearly 10% Equity St...,https://www.wsj.com/tech/trump-to-announce-u-s...
1,2025-08-22-18-51-52 +0000,wapo,Hurricane Erin tracker: Map and projected stor...,https://www.washingtonpost.com/weather/2025/08...
2,2025-08-22-18-49-42 +0000,nyt,A Bear Walked Into an Ice Cream Parlor. The St...,https://www.nytimes.com/2025/08/22/us/bear-lak...
3,2025-08-22-18-46-49 +0000,nyt,Howard Stern Helped Make SiriusXM Billions. Ca...,https://www.nytimes.com/2025/08/22/business/me...
4,2025-08-22-18-46-09 +0000,nyt,Nvidia Is Making a New Chip for China Amid Deb...,https://www.nytimes.com/2025/08/22/business/je...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,63
32,new,21
143,court,12
387,cut,11
802,erik,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...,196
82,2025-08-22-16-19-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...,128
37,2025-08-22-17-46-20 +0000,nypost,How John Bolton’s controversial 2020 memoir sp...,https://nypost.com/2025/08/22/us-news/how-john...,122
117,2025-08-22-14-55-00 +0000,wsj,A federal judge ordered the government to dism...,https://www.wsj.com/politics/policy/federal-ju...,115
296,2025-08-21-22-11-40 +0000,latimes,California voters will decide redistricting in...,https://www.latimes.com/california/story/2025-...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,196,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...
10,79,2025-08-22-18-33-00 +0000,wsj,Canada’s decision to drop tariffs on most U.S....,https://www.wsj.com/articles/canadas-rollback-...
274,67,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
54,55,2025-08-22-17-03-14 +0000,nypost,Controversial NYC judge who flashed firearm in...,https://nypost.com/2025/08/22/us-news/controve...
223,54,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
162,50,2025-08-22-11-07-29 +0000,nypost,"Putin demands Ukraine surrender Donbas, give u...",https://nypost.com/2025/08/22/world-news/putin...
168,49,2025-08-22-10-45-16 +0000,nypost,Israel warns Hamas that ‘gates of hell’ will o...,https://nypost.com/2025/08/22/world-news/israe...
117,43,2025-08-22-14-55-00 +0000,wsj,A federal judge ordered the government to dism...,https://www.wsj.com/politics/policy/federal-ju...
102,40,2025-08-22-15-34-58 +0000,wapo,Powell says weaker labor market may merit rate...,https://www.washingtonpost.com/business/2025/0...
153,38,2025-08-22-12-05-57 +0000,wapo,He agreed to conduct a band when he turned 100...,https://www.washingtonpost.com/lifestyle/2025/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
